In [1]:
import extended_cir
import pandas as pd
file_path = "data.xlsx"
ir_termS = pd.read_excel(file_path, sheet_name='interest_rate_term_structure')
cr_termS = pd.read_excel(file_path, sheet_name='credit_risk_term_structure')
ir_params = pd.read_excel(file_path, sheet_name='interest_rate_parameters')
cr_params = pd.read_excel(file_path, sheet_name='credit_risk_parameters')
cr_model = extended_cir.CreditRiskCIRPP(ir_termS, cr_termS, cr_params['k'][0], cr_params['theta'][0],
                                        cr_params['sigma'][0], cr_params['x0'][0], cr_params['recovery_rate'][0], cr_params['premium_frequency'][0])

In [2]:
def A(t, T, params):
    k, theta, sigma, x0 = params
    h = np.sqrt(np.square(k) + 2 * np.square(sigma))
    tau = T - t
    power = 2 * k * theta / sigma ** 2
    denominator = 2 * h + (k + h) * (np.exp(tau * h) - 1)
    numerator = 2 * h * (np.exp((k + h) * tau / 2))
    return (numerator / denominator) ** power

def B(t, T, params):
    k, theta, sigma, x0 = params
    h = np.sqrt(np.square(k) + 2 * np.square(sigma))
    tau = T - t
    numerator = 2 * (np.exp(tau * h) - 1)
    denominator = 2 * h + (k + h) * (np.exp(tau * h) - 1)
    return numerator / denominator

def pcir(t, T, xt, params):
    return A(t, T, params) * np.exp(-B(t, T, params) * xt)

def fcir(t, params):
    k, theta, sigma, x0 = params
    return np.log(pcir(0, t, x0, params))

def omega(t, spline, params):
    return spline(t) + fcir(t, params)

def omega_2(t, spline, params):
    return (spline(t) + fcir(t, params))**2

In [5]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
def func_minimize(params):
    g = lambda x: omega_2(x, cr_model.termStructureSpline, params)
    return quad(g, 0, 10)[0]

def constras(params, t):
    return omega(t, cr_model.termStructureSpline, params)

def post_constraint(params):
    k, theta, sigma, x0 = params
    return 2*k*theta - sigma**2

constraints = []
difft= 20/12
for j in np.arange(2*difft, 60/12, difft):
    constraints.append({'type': 'ineq', 'fun': lambda params, i=j: constras(params, i) - constras(params, i - difft)})
constraints.append({'type': 'ineq', 'fun': post_constraint})

initial_guess = [0.354201,0.00121853,0.00238186,0.0181]
bounds = [(1e-1, 0.6), (1e-3, 0.01), (1e-3, 0.01), (1e-4, 0.1)]
# call the minimization function with constraints
result = minimize(func_minimize, initial_guess, method='SLSQP', constraints=constraints, bounds=bounds)

# print the result
if result.success:
    fitted_params = result.x
    print('Fitted parameters:', fitted_params)
else:
    raise ValueError(result.message)

Fitted parameters: [0.35590523 0.00149927 0.00237648 0.00197182]


In [4]:
# ValueError: Positive directional derivative for linesearch
# ValueError: Inequality constraints incompatible
# ValueError: Iteration limit reached